In [ ]:
import tensorflow as tf
import os
# os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [ ]:
class Config:
  DATASET_PATH ="/content/drive/MyDrive/UCSD_Anomaly_Dataset.v1p2/UCSDped2/Train"
  SINGLE_TEST_PATH = "/content/drive/MyDrive/UCSD_Anomaly_Dataset.v1p2/UCSDped2/Train/Train007"
  BATCH_SIZE = 4
  EPOCHS = 40
  MODEL_PATH = "/content/drive/MyDrive/model.hdf5"
  TEST_SIZE = 0.2

In [ ]:
import tensorflow as tf

# Check if GPU is available
gpu_available = tf.test.is_gpu_available()

print("GPU available:", gpu_available)


In [ ]:
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {} '.format(device_name))

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
from os import listdir
from os.path import isfile, join, isdir
import numpy as np
from PIL import Image

from os import listdir
from os.path import isfile, join, isdir
import numpy as np
from PIL import Image

def get_clips_by_stride(stride, frames_list, sequence_size):
    """For data augmenting purposes."""
    clips = []
    sz = len(frames_list)
    clip = np.zeros(shape=(sequence_size, 256, 256, 1))
    cnt = 0
    for start in range(0, stride):
        for i in range(start, sz, stride):
            clip[cnt, :, :, 0] = frames_list[i]
            cnt = cnt + 1
            if cnt == sequence_size:
                clips.append(np.copy(clip))
                cnt = 0
    return clips

def get_dataset():
    """Prepare the dataset."""
    all_sequences = []
    # Loop over the training folders (Train000, Train001, ...)
    for f in sorted(listdir(Config.DATASET_PATH)):
        directory_path = join(Config.DATASET_PATH, f)
        if isdir(directory_path):
            all_frames = []
            # Loop over all the images in the folder (0.tif, 1.tif, ..., 199.tif)
            for c in sorted(listdir(directory_path)):
                img_path = join(directory_path, c)
                if str(img_path)[-3:] == "tif":
                    img = Image.open(img_path).resize((256, 256))
                    img = np.array(img, dtype=np.float32) / 256.0
                    all_frames.append(img)
            # Get the 10-frame sequences from the list of images after applying data augmentation
            for stride in range(1, 3):
                all_sequences.extend(get_clips_by_stride(stride=stride, frames_list=all_frames, sequence_size=10))
    return all_sequences

In [ ]:
pip install keras_layer_normalization

In [ ]:
import keras
from keras.layers import Conv2DTranspose, ConvLSTM2D, BatchNormalization, TimeDistributed, Conv2D
from keras.models import Sequential, load_model
from keras_layer_normalization import LayerNormalization

def get_model(reload_model=True):
    with tf.device('/device:GPU:0'):
      """Load or retrain the model."""
      if not reload_model:
          return load_model(Config.MODEL_PATH, custom_objects={'LayerNormalization': LayerNormalization})
      training_set = get_dataset()
      training_set = np.array(training_set)
      seq = Sequential()
      seq.add(TimeDistributed(Conv2D(128, (4, 4), strides=2, padding="same"), batch_input_shape=(None, 10, 256, 256, 1)))
      seq.add(LayerNormalization())
      seq.add(TimeDistributed(Conv2D(64, (2, 2), strides=2, padding="same")))
      seq.add(LayerNormalization())
      seq.add(TimeDistributed(Conv2D(32, (3, 3), strides=2, padding="same")))
      seq.add(LayerNormalization())
      seq.add(ConvLSTM2D(64, (3, 3), padding="same", return_sequences=True))
      seq.add(LayerNormalization())
      seq.add(ConvLSTM2D(32, (3, 3), padding="same", return_sequences=True))
      seq.add(LayerNormalization())
      seq.add(ConvLSTM2D(64, (3, 3), padding="same", return_sequences=True))
      seq.add(LayerNormalization())
      seq.add(TimeDistributed(Conv2DTranspose(32, (3, 3), strides=2, padding="same")))
      seq.add(LayerNormalization())
      seq.add(TimeDistributed(Conv2DTranspose(64, (2, 2), strides=2, padding="same")))
      seq.add(LayerNormalization())
      seq.add(TimeDistributed(Conv2DTranspose(128, (4, 4), strides=2, padding="same")))
      seq.add(LayerNormalization())
      seq.add(TimeDistributed(Conv2D(1, (4, 4), activation="sigmoid", padding="same")))
      print(seq.summary())
      optimizer = keras.optimizers.Adam(learning_rate=1e-4, epsilon=1e-6)
      seq.compile(loss='mse', optimizer=optimizer)
      print("Compiling done")
      seq.fit(training_set, training_set,
              batch_size=Config.BATCH_SIZE, epochs=Config.EPOCHS, shuffle=False)
      seq.save(Config.MODEL_PATH)
      print("Training Done")
    return seq

In [ ]:
def get_single_test():
    """Load single test data."""
    sz = 200
    test = np.zeros(shape=(sz, 256, 256, 1))
    cnt = 0
    for f in sorted(listdir(Config.SINGLE_TEST_PATH)):
        if str(join(Config.SINGLE_TEST_PATH, f))[-3:] == "tif":
            img = Image.open(join(Config.SINGLE_TEST_PATH, f)).resize((256, 256))
            img = np.array(img, dtype=np.float32) / 256.0
            test[cnt, :, :, 0] = img
            cnt = cnt + 1
    return test

In [ ]:
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

def evaluate():
    """Evaluate the model."""
    model = get_model(True)
    print("Got model")
    test = get_single_test()
    print("Got test")
    sz = test.shape[0] - 10
    sequences = np.zeros((sz, 10, 256, 256, 1))
    # Apply the sliding window technique to get the sequences
    for i in range(0, sz):
        clip = np.zeros((10, 256, 256, 1))
        for j in range(0, 10):
            clip[j] = test[i + j, :, :, :]
        sequences[i] = clip

    # Get the reconstruction cost of all the sequences
    reconstructed_sequences = model.predict(sequences, batch_size=4)
    sequences_reconstruction_cost = np.array([np.linalg.norm(np.subtract(sequences[i],reconstructed_sequences[i])) for i in range(0, sz)])
    sa = (sequences_reconstruction_cost - np.min(sequences_reconstruction_cost)) / np.max(sequences_reconstruction_cost)
    sr = 1.0 - sa

    # Plot the regularity scores
    plt.plot(sr)
    plt.ylabel('Regularity score Sr(t)')
    plt.xlabel('Frame t')
    plt.show()


# Get the dataset
all_sequences = np.array(get_dataset())

# Split the dataset into training and testing data
train_data, test_data = train_test_split(all_sequences, test_size=Config.TEST_SIZE, shuffle=False)

evaluate()


